In [1]:
import pandas as pd
from sklearn import svm
from sklearn.model_selection import GridSearchCV
import os
import matplotlib.pyplot as plt
from skimage.transform import resize
from skimage.io import imread
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix
import pickle
import mahotas as mt

In [2]:
#Categories=['2','3','4-4a','5-4b','6-4c','7-5']
Categories=['4-4a','5-4b','6-4c']
print("Type y to give categories or type n to go with classification of", Categories);

#while(True):
#  check=input()
#  if(check=='n' or check=='y'):
#    break
#  print("Please give a valid input (y/n)")
#if(check=='y'):
#  print("Enter How Many types of Images do you want to classify")
#  n=int(input())
#  Categories=[]
#  print(f'please enter {n} names')
#  for i in range(n):
#    name=input()
#    Categories.append(name)
#  print(f"If not drive Please upload all the {n} category images in google collab with the same names as given in categories")

Type y to give categories or type n to go with classification of ['4-4a', '5-4b', '6-4c']


In [3]:
def extract_features(image):
        # calculate haralick texture features for 4 types of adjacency
        textures = mt.features.haralick(image)

        # take the mean of it and return it
        ht_mean = textures.mean(axis=0)
        return ht_mean

In [4]:
flat_data_arr=[]
target_arr=[]
#please use datadir='/content' if the files are upload on to google collab
#else mount the drive and give path of the parent-folder containing all category images folders.
datadir='..\Datasets\Colombiam\\svm\\thyroid-crop-small-background-square-160-tri\\'
for i in Categories:
  print(f'loading... category : {i}')
  path=os.path.join(datadir,i)
  for img in os.listdir(path):
    img_array=imread(os.path.join(path,img))
    #img_resized=resize(img_array,(150,150,3))
    #flat_data_arr.append(img_array.flatten())
    flat_data_arr.append(extract_features(img_array))
    target_arr.append(Categories.index(i))
  print(f'loaded category:{i} successfully')
flat_data=np.array(flat_data_arr)
target=np.array(target_arr)
df=pd.DataFrame(flat_data)
df['Target']=target
df

loading... category : 4-4a
loaded category:4-4a successfully
loading... category : 5-4b
loaded category:5-4b successfully
loading... category : 6-4c
loaded category:6-4c successfully


,0,1,2,3,4,5,6,7,8,9,10,11,12,Target
0,0.001317,262.755565,0.718393,466.788577,0.153522,128.135957,1604.398743,7.269827,11.724108,0.000578,4.659052,-0.185744,0.869070,0
1,0.001677,333.743065,0.750962,671.835709,0.183362,118.686824,2353.599769,7.347760,11.710716,0.000412,4.777371,-0.212103,0.917067,0
2,0.000771,407.086779,0.846236,1323.320296,0.148471,185.654325,4886.194403,7.992184,12.506104,0.000406,4.902032,-0.250309,0.959365,0
3,0.000810,325.936161,0.888130,1457.126161,0.157681,192.780041,5502.568483,8.030024,12.391644,0.000419,4.761206,-0.268674,0.970287,0
4,0.000959,416.799680,0.737602,793.897981,0.140041,218.526368,2758.792245,7.667032,12.337473,0.000443,4.931857,-0.200106,0.900531,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330,0.000948,263.963203,0.878170,1083.398436,0.158672,157.770300,4069.630541,7.819155,12.147076,0.000448,4.646734,-0.253069,0.959343,2
331,0.000853,445.482646,0.844464,1431.451560,0.139534,168.018885,5280.323596,7.940350,12.529336,0.000393,4.996057,-0.235562,0.952357,2
332,0.000840,349.050652,0.873011,1374.554724,0.150024,164.529878,5149.168244,7.971775,12.411376,0.000410,4.818975,-0.254589,0.963434,2
333,0.000847,307.145339,0.890946,1410.592201,0.160958,176.061314,5335.223467,8.001557,12.309074,0.000428,4.691495,-0.272767,0.969462,2


In [5]:
x=df.iloc[:,:-1]
y=df.iloc[:,-1]
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.30,random_state=77,stratify=y)
print('Splitted Successfully')

Splitted Successfully


In [6]:
param_grid={'C':[0.1,1,10,100,1000,10000],'gamma':[0.0001,0.001,0.1,1],'kernel':['rbf']}
#param_grid={'C':[0.1,1,10,100,1000,10000]}
svc=svm.SVC(probability=True) # internally uses 5-fold cross-validation multiclass support is handled according to a one-vs-one scheme
#svc=svm.LinearSVC() # multiclass support is handled according to a one-vs-the-rest scheme
print("The training of the model is started, please wait for while as it may take few minutes to complete")
model=GridSearchCV(svc,param_grid)
model.fit(x_train,y_train)
print('The Model is trained well with the given images')
model.best_params_

The training of the model is started, please wait for while as it may take few minutes to complete


In [ ]:
y_pred=model.predict(x_test)
print("The predicted Data is :")
y_pred

The predicted Data is :


array([4, 5, 4, 5, 2, 3, 0, 5, 2, 4, 2, 4, 5, 5, 2, 0, 5, 3, 3, 2, 4, 2,
       3, 3, 3, 2, 3, 4, 3, 2, 3, 4, 0, 2, 3, 2, 3, 5, 3, 4, 4, 4, 5, 2,
       3, 4, 0, 2, 2, 2, 2, 4, 2, 4, 3, 3, 2, 4, 3, 5, 2, 3, 2, 2, 3, 3,
       5, 2, 3, 3, 4, 5, 2, 0, 3, 2, 0, 4, 3, 3, 2, 3, 2, 5, 0, 2, 2, 3,
       4, 3, 5, 3, 3, 5, 0, 2, 2, 5, 2, 0, 5, 5, 2, 2, 2, 3, 3, 2, 4, 4,
       0, 3, 5, 3, 3, 0, 4, 0, 3, 2, 3, 0, 5, 2, 2, 3, 3, 2, 4, 4, 3, 0,
       2, 0, 3, 3, 3, 2])

In [ ]:
print("The actual data is:")
np.array(y_test)

The actual data is:


array([4, 5, 4, 0, 2, 4, 2, 2, 1, 2, 2, 0, 3, 5, 2, 2, 5, 3, 1, 2, 0, 3,
       2, 2, 4, 2, 2, 5, 0, 2, 2, 3, 0, 3, 5, 5, 2, 3, 5, 4, 3, 3, 3, 5,
       4, 0, 3, 2, 4, 3, 0, 3, 3, 3, 2, 4, 0, 3, 0, 3, 2, 3, 5, 2, 2, 4,
       2, 2, 2, 5, 2, 4, 3, 2, 0, 4, 2, 1, 4, 3, 2, 5, 4, 4, 2, 2, 1, 2,
       5, 3, 2, 4, 4, 4, 4, 2, 0, 2, 4, 2, 3, 5, 4, 0, 5, 0, 3, 3, 0, 3,
       2, 2, 5, 3, 3, 2, 4, 1, 4, 1, 4, 0, 5, 3, 3, 4, 2, 3, 3, 4, 4, 3,
       2, 3, 4, 2, 3, 2])

In [ ]:
#classification_report(y_pred,y_test)
print(f"The model is {accuracy_score(y_pred,y_test)*100}% accurate")
confusion_matrix(y_pred,y_test)

The model is 26.08695652173913% accurate


array([[ 2,  1,  8,  3,  1,  0],
       [ 0,  0,  0,  0,  0,  0],
       [ 4,  3, 15,  9,  5,  4],
       [ 4,  1, 12,  8, 13,  4],
       [ 4,  1,  2,  8,  5,  2],
       [ 1,  0,  4,  5,  3,  6]], dtype=int64)

In [11]:
pickle.dump(model,open('img_model.p','wb'))
print("Pickle is dumped successfully")

Pickle is dumped successfully
